In [8]:
pip install mysql

     ---------------------------------------- 0.0/200.1 kB ? eta -:--:--
     --------------------------------- ---- 174.1/200.1 kB 5.3 MB/s eta 0:00:01
     -------------------------------------- 200.1/200.1 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import pymysql
import mysql.connector
import plotly

In [18]:
### Table : train / ttf / fault 불러오기

db = mysql.connector.connect(
    host="limemoni-2.cfcq69qzg7mu.ap-northeast-1.rds.amazonaws.com",
    user="oneday",
    password="1234",
    database="j6database",
)
cursor = db.cursor()

# 4단계 : sql문 작성(데이터 read)
sql = "SELECT * FROM input_data;"

# 5단계 : sql문 실행(데이터 read)
cursor.execute(sql)

# 6단계 : fetch 메서드(조회결과 콘솔창에서 보기 위함)
result = cursor.fetchall() # fetchall() : 전부 가져오기

colnames = cursor.description # 변수정보
cols = [colnames[i][0] for i in range(len(colnames))] # 변수명

# 9단계 : df
df = pd.DataFrame(result, columns=cols)

# 8단계 : commit 및 연결 해제
db.commit() # 커밋은 반복할 필요 없음
db.close()

In [19]:
df

,time,Tool,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,...,ETCHPBNGASREADBACK,FIXTURETILTANGLE,ROTATIONSPEED,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION,input_time
0,3283834,01M02,1,9253,10511673,67,1,-0.079428,-0.958814,-0.959509,...,-2.783162,2.027848,-0.013207,-0.122437,0,2.395370,2.871511,3.175941,4.037785,2023-10-20 14:18:41
1,3283838,01M02,1,9253,10511673,67,1,-0.079419,-0.958928,-0.959651,...,-2.780884,2.027848,-0.013207,-0.122437,0,2.395370,2.871511,3.175941,4.037785,2023-10-20 14:18:46
2,3283842,01M02,1,9253,10511673,67,1,-0.079409,-0.959156,-0.960006,...,-2.780884,2.027848,-0.013207,-0.122437,0,2.395370,2.871511,3.175941,4.037785,2023-10-20 14:18:51
3,3283846,01M02,1,9253,10511673,67,1,-0.079420,-0.958928,-0.959297,...,-2.780884,2.027848,-0.013207,-0.122437,0,2.395370,2.871511,3.175941,4.037785,2023-10-20 14:18:56
4,3283850,01M02,1,9253,10511673,67,1,-0.079430,-0.959269,-0.959438,...,-2.784389,2.027848,-0.013207,-0.122437,0,2.395370,2.871511,3.175941,4.037785,2023-10-20 14:19:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,3287354,01M02,1,9253,10511673,67,39,-0.000342,-0.952331,-0.959297,...,0.354293,0.022131,-0.013207,-0.122437,1,2.398170,2.874643,3.179197,0.021044,2023-10-20 15:58:46
1110,3287358,01M02,1,9253,10511673,67,39,-0.000342,-0.952217,-0.959509,...,0.354293,0.022131,-0.013207,-0.122437,1,2.398173,2.874649,3.179203,0.021044,2023-10-20 15:58:53
1111,3287362,01M02,1,9253,10511673,67,39,-0.000342,-0.952274,-0.959793,...,0.354293,0.022131,-0.013207,-0.122437,1,2.398176,2.874653,3.179207,0.021044,2023-10-20 15:58:58
1112,3287366,01M02,1,9253,10511673,67,39,-0.000342,-0.952558,-0.959793,...,0.358499,0.022131,-0.013207,-0.122437,1,2.398180,2.874654,3.179209,0.021044,2023-10-20 15:59:03


In [ ]:
plotly